In [1]:
import requests


In [14]:
with open('kegg_id.txt', 'r') as file:
    keggID = file.read()
    keggID = keggID.split('\n')
    print(keggID)

def getDataFromKegg(operation, argument):
    url = f"https://rest.kegg.jp/{operation}/{argument}"

    resp = requests.get(
        url
    )

    if resp.ok:
        return resp.text
  
for keggid in keggID:
    info = getDataFromKegg('get', keggid).split('\n')
    for cos in info:
        if cos.startswith("POSITION"):
            cos = cos.strip("POSITION").strip().split(':')
            print(f'{keggid}, chromosom: {cos[0]}, pozycja: {cos[1]}')
        else:
            pass
    



['hsa:3107', 'hsa:3105', 'hsa:3106', 'hsa:3135', 'hsa:3133']
hsa:3107, chromosom: 6, pozycja: complement(31268749..31272092)
hsa:3105, chromosom: 6, pozycja: 29942532..29945870
hsa:3106, chromosom: 6, pozycja: complement(31353875..31357179)
hsa:3135, chromosom: 6, pozycja: 29826474..29831021
hsa:3133, chromosom: 6, pozycja: 30489509..30494194


In [15]:
import time

def runClustal(sequecnes_in_fasta):
    resp = requests.post(
        'https://www.ebi.ac.uk/Tools/services/rest/clustalo/run',
        data={
            "email": "marcel.thiel@ug.edu.pl",
            "sequence": sequecnes_in_fasta,
            "outfmt": "fa"
        }
    )
    jobid = resp.text

    return jobid


def getStatus(jobId):
    url = f"https://www.ebi.ac.uk/Tools/services/rest/clustalo/status/{jobId}"
    
    resp = requests.get(
        url
    )

    return resp.text


def getResults(jobId):
    url = f"https://www.ebi.ac.uk/Tools/services/rest/clustalo/result/{jobId}/fa"

    resp = requests.get(
        url
    )

    return resp.text


with open('genes_fasta', 'r') as file:
    sequencesFromFasta = file.read()

jobid = runClustal(sequencesFromFasta)

print(f"JOB ID IS: {jobid}")

max_attempt = 20

for attept in range(max_attempt):
    status = getStatus(jobid)

    if status == "FINISHED":
        results = getResults(jobid)
        print(results)

        break

    time.sleep(1)

JOB ID IS: clustalo-R20240222-192144-0122-31359896-p1m
>3107 K06751 MHC class I antigen | (RefSeq) HLA-C, D6S204, HLA-JY3, HLAC, HLC-C, MHC, PSORS1; major histocompatibility complex, class
atgcgggtcatggcgccccgagccctcctcctgctgctctcgggaggcctggccctgacc
gagacctgggcctgctcccactccatgaggtatttcgacaccgccgtgtcccggcccggc
cgcggagagccccgcttcatctcagtgggctacgtggacgacacgcagttcgtgcggttc
gacagcgacgccgcgagtccgagaggggagccgcgggcgccgtgggtggagcaggagggg
ccggagtattgggaccgggagacacagaagtacaagcgccaggcacaggctgaccgagtg
agcctgcggaacctgcgcggctactacaaccagagcgaggacgggtctcacaccctccag
aggatgtctggctgcgacctggggcccgacgggcgcctcctccgcgggtatgaccagtcc
gcctacgacggcaaggattacatcgccctgaacgaggacctgcgctcctggaccgccgcg
gacaccgcggctcagatcacccagcgcaagttggaggcggcccgtgcggcggagcagctg
agagcctacctggagggcacgtgcgtggagtggctccgcagatacctggagaacgggaag
gagacgctgcagcgcgcagaacccccaaagacacacgtgacccaccaccccctctctgac
catgaggccaccctgaggtgctgggccctgggcttctaccctgcggagatcacactgacc
tggcagcgggatggggaggaccagacccaggacaccgagcttgtggagaccaggccagca
ggagatggaaccttccag

In [30]:
def getDataFromPDB():
    url = f"https://files.rcsb.org/view/1LKX.pdb"

    resp = requests.get(url)

    if resp.ok:
        return resp.text
    
    return False

info = getDataFromPDB()

def jeden(info: str):
    infoInLines = info.split('\n')
    for cos in infoInLines:
        if cos.startswith('HEADER'):
            return cos
        else:
            pass

print(jeden(info))

def dwa(info: str):
    infoInLines = info.split('\n')
    helix = 0
    sheet = 0
    atoms = 0
    for cos in infoInLines:
        if cos.startswith("HELIX"):
            helix += 1
        elif cos.startswith("SHEET"):
            sheet += 1
        elif cos.startswith("ATOM"):
            atoms +=1
        else:
            pass

    return helix, sheet, atoms

helix, sheet, atoms = dwa(info)
print(f'helix: {helix}, sheet: {sheet}, atoms: {atoms}')



HEADER    CONTRACTILE PROTEIN                     26-APR-02   1LKX              
helix: 127, sheet: 58, atoms: 21069
